<a href="https://colab.research.google.com/github/cherryng/MLcourse/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Python packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
import pickle
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams.update({'font.size': 13,'font.family': 'serif'})


In [2]:
data_orders = pd.read_csv("orders.csv")
print(f"Number of records: {len(data_orders)}")
print(data_orders.head())

#Find out number of users:
user_id = data_orders["user_id"]
total_user = user_id.max()
print("number of users:", total_user)

Number of records: 3421083
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  
number of users: 206209


In [3]:
#For applying the model to the Kaggle test data, I need an input ("Kaggle_test") which contains the following columns:
# Index(['product_id', 'user_id', 'order_dow', 'order_hour_of_day','days_since_prior_order'],dtype='object')

#From "sample_submission", I get the "order_id"
#I need to figure out the "user_id" for each of these orders by merging with frame "data_orders"

data_samp = pd.read_csv("sample_submission.csv")
data_samp = data_samp.drop(['products'], axis=1)

print(f"Number of records: {len(data_samp)}")
Kaggle_test = pd.merge(data_samp, data_orders, on='order_id', how='left')
Kaggle_test = Kaggle_test.drop(['eval_set','order_number'], axis=1)
Kaggle_test.head



Number of records: 75000


<bound method NDFrame.head of        order_id  user_id  order_dow  order_hour_of_day  days_since_prior_order
0            17    36855          6                 15                     1.0
1            34    35220          3                 11                     8.0
2           137   187107          2                 19                    30.0
3           182   115892          0                 11                     8.0
4           257    35581          6                 23                     5.0
...         ...      ...        ...                ...                     ...
74995   3420740   195822          0                  0                     3.0
74996   3420877    15955          3                 22                     1.0
74997   3420888   162374          5                 17                     7.0
74998   3420989   109358          6                 13                    13.0
74999   3421054   153585          5                 12                     7.0

[75000 rows x 5 colum

In [7]:
data_order_prior = pd.read_csv("order_products__prior.csv")
data_order_prior = pd.merge(data_order_prior,data_orders,on='order_id', how='left')
data_order_prior.head

<bound method NDFrame.head of           order_id  product_id  add_to_cart_order  reordered  user_id  \
0                2       33120                  1          1   202279   
1                2       28985                  2          1   202279   
2                2        9327                  3          0   202279   
3                2       45918                  4          1   202279   
4                2       30035                  5          0   202279   
...            ...         ...                ...        ...      ...   
32434484   3421083       39678                  6          1    25247   
32434485   3421083       11352                  7          0    25247   
32434486   3421083        4600                  8          0    25247   
32434487   3421083       24852                  9          1    25247   
32434488   3421083        5020                 10          1    25247   

         eval_set  order_number  order_dow  order_hour_of_day  \
0           prior           

In [6]:
data_orders[data_orders["order_id"] ==2]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3355525,2,202279,prior,3,5,9,8.0


In [29]:
#For each user, I need to figure out what products he/she has bought before and the history to generate the "Kaggle_input"


for index, row in df.iterrows():
    print(row['c1'], row['c2'])
    
i=36855
#For a given user, all the stuff he/she has ever bought
user_prior = data_order_prior[data_order_prior["user_id"] == i]
user_prior =  pd.merge(user_prior,Kaggle_test, on='order_id', how='left')
user_prior.head

<bound method NDFrame.head of     order_id  product_id  add_to_cart_order  reordered  user_id_x eval_set  \
0     234692       21709                  1          1      36855    prior   
1     234692       13107                  2          0      36855    prior   
2     234692       38777                  3          0      36855    prior   
3     234692       48896                  4          0      36855    prior   
4     234692        1283                  5          0      36855    prior   
5     234692       13535                  6          0      36855    prior   
6     234692       47766                  7          1      36855    prior   
7     234692       39928                  8          0      36855    prior   
8     234692       31964                  9          0      36855    prior   
9     898818       47766                  1          0      36855    prior   
10    898818       21903                  2          0      36855    prior   
11    898818        7035          

In [35]:
#For a given user, only the last time he bought each product
user_prior_uniq = user_prior.groupby('product_id')['order_id'].aggregate('max').reset_index()
user_prior_uniq["user_id"] = i
user_prior_uniq.head

<bound method NDFrame.head of     product_id  order_id  user_id
0         1283    234692    36855
1         6291   3248434    36855
2         7035    898818    36855
3        11494    898818    36855
4        13107   3248434    36855
5        13535    234692    36855
6        15613   3248434    36855
7        16965   3248434    36855
8        18288   3248434    36855
9        21463   3248434    36855
10       21709    898818    36855
11       21903    898818    36855
12       26429   1058761    36855
13       31964    234692    36855
14       38777   3248434    36855
15       39275   1058761    36855
16       39928    234692    36855
17       40002   3248434    36855
18       44056   3248434    36855
19       47766    898818    36855
20       48896    234692    36855>

In [52]:
tmp4 = pd.merge(user_prior_uniq, user_prior, how='left',on=['order_id','product_id'])
tmp4 = tmp4.drop(['order_id','user_id_y','eval_set','order_number','add_to_cart_order','reordered','user_id_x','order_dow_y','order_hour_of_day_y','days_since_prior_order_y'], axis=1)
tmp4.columns = ['product_id', 'user_id', 'order_dow', 'order_hour_of_day','days_since_prior_order']

tmp4

,product_id,user_id,order_dow,order_hour_of_day,days_since_prior_order
0,1283,36855,0,9,30.0
1,6291,36855,6,14,6.0
2,7035,36855,1,13,NaN
3,11494,36855,1,13,NaN
4,13107,36855,6,14,6.0
5,13535,36855,0,9,30.0
6,15613,36855,6,14,6.0
7,16965,36855,6,14,6.0
8,18288,36855,6,14,6.0
9,21463,36855,6,14,6.0


<bound method NDFrame.head of     product_id  order_id
0         1283    234692
1         6291   3248434
2         7035    898818
3        11494    898818
4        13107   3248434
5        13535    234692
6        15613   3248434
7        16965   3248434
8        18288   3248434
9        21463   3248434
10       21709    898818
11       21903    898818
12       26429   1058761
13       31964    234692
14       38777   3248434
15       39275   1058761
16       39928    234692
17       40002   3248434
18       44056   3248434
19       47766    898818
20       48896    234692>

In [22]:
data_order_prior[ (data_order_prior["product_id"] ==1283) & (data_order_prior["user_id"] ==36855) ]
#data_order_prior[ (data_order_prior["product_id" ==2]) & (data_order_prior["user_id" ==36855]) ]

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2225208,234692,1283,5,0,36855,prior,2,0,9,30.0
